## Read data

demonstration data set from the UEA collection

In [2]:
import numpy as np

data = [np.load(f'../data/GestureMidAirD1/{variable}_{set_name}.npy')
        for variable in ['X', 'y'] for set_name in ['train', 'test']]

X_train, X_test, y_train, y_test = data

print("X_train dims: ", X_train.shape)
print("X_test dims: ", X_test.shape)

X_train dims:  (208, 1, 360)
X_test dims:  (130, 1, 360)


in order to apply the foundation model to your data, `X_train` and `X_test` should be of the shape `(n_samples, n_channels=1, seq_len)`, where `seq_len` is a multiple of 32

if original sequence length is different, resize it, for example, using the following function:

In [4]:
import torch
import torch.nn.functional as F

def resize(X):
    X_scaled = F.interpolate(torch.tensor(X, dtype=torch.float), size=512, mode='linear', align_corners=False)
    return X_scaled.numpy()
    
X_train, X_test = resize(X_train), resize(X_test)

print("X_train dims: ", X_train.shape)
print("X_test dims: ", X_test.shape)

X_train dims:  (208, 1, 512)
X_test dims:  (130, 1, 512)


## Load model

In [14]:
from mantis.architecture import Mantis8M
    
device = 'cpu' # set device
network = Mantis8M(device=device) # init model
network = network.from_pretrained("paris-noah/Mantis-8M") # load weights

## Fine-tune the network using MantisTrainer

initialize the trainer and some arguments to pass during fine-tuning

In [15]:
from mantis.trainer import MantisTrainer

model = MantisTrainer(device=device, network=network)

# initialize some training parameters
def init_optimizer(params): return torch.optim.AdamW(
    params, lr=2e-4, betas=(0.9, 0.999), weight_decay=0.05)

### Fine-tuning a classification head

In [18]:
fine_tuning_type = 'head'

# fine-tune the model
model.fit(X_train, y_train, num_epochs=100,
            fine_tuning_type=fine_tuning_type, init_optimizer=init_optimizer)

# predict labels
y_pred = model.predict(X_test)

Epoch 99: Train Loss 2.5544: 100%|████████████████████████████████████████████████████| 100/100 [00:05<00:00, 17.17it/s]


In [19]:
# evaluate performance
print(f'Accuracy on the test set is {np.mean(y_test == y_pred)}')

Accuracy on the test set is 0.35384615384615387


### Full fine-tuning

In [20]:
fine_tuning_type = 'full'

# fine-tune the model
model.fit(X_train, y_train, num_epochs=100,
            fine_tuning_type=fine_tuning_type, init_optimizer=init_optimizer)

# predict labels
y_pred = model.predict(X_test)

Epoch 99: Train Loss 0.1134: 100%|████████████████████████████████████████████████████| 100/100 [00:06<00:00, 15.82it/s]


In [21]:
# evaluate performance
print(f'Accuracy on the test set is {np.mean(y_test == y_pred)}')

Accuracy on the test set is 0.7846153846153846
